In [1]:
#| include: false

import duckdb
import pandas as pd
%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

%sql CREATE OR REPLACE TABLE user_actions AS SELECT * FROM read_csv('00_data/sql/user_actions.csv', header=True, columns={'user_id': 'INT', 'order_id': 'INT', 'action': 'VARCHAR', 'time': 'TIMESTAMP'}, timestampformat='%d/%m/%y %H:%M');
%sql CREATE OR REPLACE TABLE courier_actions AS SELECT * FROM read_csv('00_data/sql/courier_actions.csv', header=True, columns={'courier_id': 'INT', 'order_id': 'INT', 'action': 'VARCHAR', 'time': 'TIMESTAMP'}, timestampformat='%d/%m/%y %H:%M');
%sql CREATE OR REPLACE TABLE orders AS SELECT * FROM read_csv('00_data/sql/orders.csv', header=True, columns={'order_id': 'INT', 'creation_time': 'TIMESTAMP', 'product_ids': 'INT[]'}, timestampformat='%d/%m/%y %H:%M');
%sql CREATE OR REPLACE TABLE users AS SELECT * FROM read_csv('00_data/sql/users.csv', header=True, columns={'user_id': 'INT', 'birth_date': 'DATE', 'sex': 'VARCHAR'}, dateformat='%d/%m/%y');
%sql CREATE OR REPLACE TABLE couriers AS SELECT * FROM read_csv('00_data/sql/couriers.csv', header=True, columns={'courier_id': 'INT', 'birth_date': 'DATE', 'sex': 'VARCHAR'}, dateformat='%d/%m/%y');
%sql CREATE OR REPLACE TABLE products AS SELECT * FROM read_csv('00_data/sql/products.csv', header=True, columns={'product_id': 'INT', 'name': 'VARCHAR', 'price': 'DOUBLE'});

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


,Count
0,87


# Аналіз продуктових метрик

Минулого разу ми навчилися будувати повноцінні дашборди і представляти результати аналізу у зрозумілій бізнесі формі — це дуже важлива навичка, необхідна будь-якому аналітику.

Цього разу ми трохи докладніше поговоримо про аналіз найважливіших метрик, які дозволяють з різних боків оцінити те, наскільки добре працює наш продукт.

Насправді раніше ми вже рахували окремі показники — наприклад, метрики залучення користувачів **DAU**, **WAU** та **MAU**, кількість замовлень, частку скасованих замовлень тощо. Це дійсно важливі показники, але часто представники бізнесу в першу чергу звертають увагу на більш зрозумілі показники, виражені в грошовій формі. Це можуть бути виручка, витрати чи рентабельність, а також відносні показники на кшталт доходу на одного користувача та середнього чека – все це теж важливо вміти розраховувати, чим ми з вами й займемося.

Крім того, наприкінці ми обговоримо методику розрахунку ще одного важливого показника – **Retention rate**.

На цей раз вам знову буде запропоновано написати кілька SQL-запитів і візуалізувати їх результат за допомогою графіків.

## Задача 1

Почнемо з виручки - найбільш загального показника, який покаже, який дохід приносить наш сервіс.

::::: {.callout-note icon=false}
## Завдання
:::: {#exr-sql-metrics-01}
<br>
Для кожного дня в таблиці `orders` розрахуйте такі показники:

1. Виручку, одержану в цей день.
2. Сумарний виторг на поточний день.
3. Приріст виручки, отриманої цього дня, щодо значення виручки за попередній день.

Колонки з показниками назвіть відповідно `revenue`, `total_revenue`, `revenue_change`. Колонку з датами назвіть `date`.

Приріст виручки розрахуйте у відсотках та округліть значення **до двох знаків після коми**.

Результат має бути відсортований за зростанням дати.

Поля в результуючій таблиці: `date`, `revenue`, `total_revenue`, `revenue_change`

**Пояснення:**

Вважатимемо, що оплата за замовлення надходить відразу після його оформлення, тобто випадки, коли замовлення було оформлено в один день, а оплата отримана наступного, виникнути не можуть.

Сумарна виручка на поточний день – це результат складання виручки, отриманої поточного дня, зі значеннями аналогічного показника всіх попередніх днів.

При розрахунку виручки пам'ятайте, що не всі замовлення були сплачені, деякі були скасовані користувачами.

Не забувайте при діленні заздалегідь переводити значення до потрібного типу даних. Пропущені значення приросту для першої дати не заповнюйте - просто залиште поля в цьому рядку порожніми.

::: {.callout-note collapse="true"}
## Підказка на випадок, якщо зовсім не виходить

Для вирішення завдання вам знадобиться інформація про замовлення з таблиці `orders` та ціни на товари з таблиці `products`. Щоб порахувати виторг для кожного дня, спочатку необхідно порахувати вартість кожного замовлення. Це можна зробити, склавши ціни товарів, що входять на замовлення. Щоб правильно приєднати дані про ціни на товари, списки із вмістом замовлень потрібно попередньо розширити за допомогою функції `unnest`. Після того як для кожного дня буде порахована сумарна вартість усіх замовлень (виручка), за допомогою віконних функцій можна порахувати суму наростаючим підсумком (загальну виручку) та приріст виручки (різницю між виручкою в поточний день та виручкою в попередній день, поділену на виручку в попередній день).
:::


::::
:::::

In [2]:
#| code-fold: true
#| code-summary: "Рішення"

%%sql
SELECT date,
       revenue,
       sum(revenue) OVER (ORDER BY date) as total_revenue,
       round(100 * (revenue - lag(revenue, 1) OVER (ORDER BY date))::decimal / lag(revenue, 1) OVER (ORDER BY date),
             2) as revenue_change
FROM   (SELECT creation_time::date as date,
               sum(price) as revenue
        FROM   (SELECT creation_time,
                       unnest(product_ids) as product_id
                FROM   orders
                WHERE  order_id not in (SELECT order_id
                                        FROM   user_actions
                                        WHERE  action = 'cancel_order')) t1
            LEFT JOIN products using (product_id)
        GROUP BY date) t2

,date,revenue,total_revenue,revenue_change
0,2022-08-24,49924.0,49924.0,NaN
1,2022-08-25,430860.0,480784.0,763.03
2,2022-08-26,534766.0,1015550.0,24.12
3,2022-08-27,817053.0,1832603.0,52.79
4,2022-08-28,1133370.0,2965973.0,38.71
5,2022-08-29,1279891.0,4245864.0,12.93
6,2022-08-30,1279377.0,5525241.0,-0.04
7,2022-08-31,1312720.0,6837961.0,2.61
8,2022-09-01,1406101.0,8244062.0,7.11
9,2022-09-02,1907107.0,10151169.0,35.63


Запишемо результат запиту у змінну `results_1`.

In [3]:
#| code-fold: true

%%sql
results_1 << SELECT date,
       revenue,
       sum(revenue) OVER (ORDER BY date) as total_revenue,
       round(100 * (revenue - lag(revenue, 1) OVER (ORDER BY date))::decimal / lag(revenue, 1) OVER (ORDER BY date),
             2) as revenue_change
FROM   (SELECT creation_time::date as date,
               sum(price) as revenue
        FROM   (SELECT creation_time,
                       unnest(product_ids) as product_id
                FROM   orders
                WHERE  order_id not in (SELECT order_id
                                        FROM   user_actions
                                        WHERE  action = 'cancel_order')) t1
            LEFT JOIN products using (product_id)
        GROUP BY date) t2

Побудуємо візуалізацію за отриманими даними:

In [4]:
# | label: fig-sql-metrics-01
# | fig-cap: "Графік за результатами SQL-запиту"
# | fig-subcap:
# |   - "Динаміка щоденної виручки"
# |   - "Динаміка загальної виручки"
# | column: screen-inset-shaded
# | layout-nrow: 1

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Bar(
        name="revenue_change",
        x=results_1.date,
        y=results_1.revenue_change,
        offsetgroup=1,
    ),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(name="revenue", x=results_1.date, y=results_1.revenue, offsetgroup=2),
    secondary_y=True,
)
fig.update_yaxes(rangemode="tozero", secondary_y=True)
fig.update_layout(
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
)

fig.show()

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=results_1.date,
        y=results_1.total_revenue,
        mode="lines+markers",
        name="total_revenue",
    )
)
fig.update_layout(
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)
fig.show()

Проаналізуйте побудовані графіки та спробуйте відповісти на такі питання:

1. Які дні спостерігалося помітне зниження щоденної виручки?
2. Із чим це могло бути пов'язане?

## Задача 2

Тепер на основі даних про виторг розрахуємо кілька відносних показників, які покажуть, скільки в середньому споживачі готові платити за послуги нашого сервісу доставки. Зупинимося на наступних метриках:

1. **ARPU (Average Revenue Per User)** - середня виручка на одного користувача за певний період.
2. **ARPPU (Average Revenue Per Paying User)** - середня виручка на одного користувача, що оплачує замовлення за певний період.
3. **AOV (Average Order Value)** - середній чек або відношення виручки за певний період до загальної кількості замовлень за цей час.

Якщо за період, що розглядається, сервіс заробив 100 000 грошових одиниць і при цьому ним користувалися 500 унікальних користувачів, з яких 400 зробили загалом 650 замовлень, тоді метрики будуть мати наступні значення:

$$ARPU = 100000/500 = 200$$

$$ARPPU = 100 000/400 = 250$$

$$AOV=100000/650≈153,85$$

::::: {.callout-note icon=false}
## Завдання
:::: {#exr-sql-metrics-02}
<br>
Для кожного дня в таблицях `orders` та `user_actions` розрахуйте такі показники:

1. Виручку користувача (**ARPU**) за поточний день.
2. Виручку на користувача, що платить (**ARPPU**) за поточний день.
3. Виручку із замовлення, або середній чек (**AOV**) за поточний день.

Колонки з показниками назвіть відповідно `arpu`, `arppu`, `aov`. Колонку з датами назвіть `date`.

Під час розрахунку всіх показників округляйте значення **до двох знаків після коми**.

Результат має бути відсортований за зростанням дати.

Поля в результуючій таблиці: `date`, `arpu`, `arppu`, `aov`

**Пояснення:**

Вважатимемо, що оплата за замовлення надходить відразу після його оформлення, тобто випадки, коли замовлення було оформлено в один день, а оплата отримана наступного, виникнути не можуть.

Користувачами, що оплатили замовлення будемо вважати тих, які в даний день оформили хоча б одне замовлення, яке надалі не було скасовано.

При розрахунку виручки пам'ятайте, що не всі замовлення були сплачені, деякі були скасовані користувачами.

Не забувайте при діленні заздалегідь приводити значення до потрібного типу даних.

::: {.callout-note collapse="true"}
## Підказка на випадок, якщо зовсім не виходить

Для вирішення завдання необхідно спочатку для кожного дня порахувати виручку, кількість всіх користувачів, кількість користувачів, що сплатили замовлення і кількість замовлень. Потім необхідно об'єднати отримані таблиці та розрахувати всі необхідні відносні показники. Виручку ми вже рахували у минулому завданні.
:::


::::
:::::

In [5]:
#| code-fold: true
#| code-summary: "Рішення"

%%sql
SELECT date,
       round(revenue::decimal / users, 2) as arpu,
       round(revenue::decimal / paying_users, 2) as arppu,
       round(revenue::decimal / orders, 2) as aov
FROM   (SELECT creation_time::date as date,
               count(distinct order_id) as orders,
               sum(price) as revenue
        FROM   (SELECT order_id,
                       creation_time,
                       unnest(product_ids) as product_id
                FROM   orders
                WHERE  order_id not in (SELECT order_id
                                        FROM   user_actions
                                        WHERE  action = 'cancel_order')) t1
            LEFT JOIN products using(product_id)
        GROUP BY date) t2
    LEFT JOIN (SELECT time::date as date,
                      count(distinct user_id) as users
               FROM   user_actions
               GROUP BY date) t3 using (date)
    LEFT JOIN (SELECT time::date as date,
                      count(distinct user_id) as paying_users
               FROM   user_actions
               WHERE  order_id not in (SELECT order_id
                                       FROM   user_actions
                                       WHERE  action = 'cancel_order')
               GROUP BY date) t4 using (date)
ORDER BY date

,date,arpu,arppu,aov
0,2022-08-24,372.57,393.10,361.77
1,2022-08-25,508.09,525.44,406.86
2,2022-08-26,452.04,470.33,369.57
3,2022-08-27,509.38,527.81,381.62
4,2022-08-28,528.38,544.10,378.04
5,2022-08-29,559.15,581.24,391.76
6,2022-08-30,546.74,567.85,379.52
7,2022-08-31,517.63,540.21,384.96
8,2022-09-01,499.33,518.86,381.26
9,2022-09-02,537.67,556.17,381.35


Збережемо результат запиту у змінну `results_2`.

In [6]:
#| code-fold: true

%%sql
results_2 << SELECT date,
       round(revenue::decimal / users, 2) as arpu,
       round(revenue::decimal / paying_users, 2) as arppu,
       round(revenue::decimal / orders, 2) as aov
FROM   (SELECT creation_time::date as date,
               count(distinct order_id) as orders,
               sum(price) as revenue
        FROM   (SELECT order_id,
                       creation_time,
                       unnest(product_ids) as product_id
                FROM   orders
                WHERE  order_id not in (SELECT order_id
                                        FROM   user_actions
                                        WHERE  action = 'cancel_order')) t1
            LEFT JOIN products using(product_id)
        GROUP BY date) t2
    LEFT JOIN (SELECT time::date as date,
                      count(distinct user_id) as users
               FROM   user_actions
               GROUP BY date) t3 using (date)
    LEFT JOIN (SELECT time::date as date,
                      count(distinct user_id) as paying_users
               FROM   user_actions
               WHERE  order_id not in (SELECT order_id
                                       FROM   user_actions
                                       WHERE  action = 'cancel_order')
               GROUP BY date) t4 using (date)
ORDER BY date

Візуалізуємо отримані дані:

In [7]:
# | label: fig-sql-analytic-02
# | fig-cap: "Динаміка ARPU, ARPPU та AOV"

fig = go.Figure()
fig.add_trace(
    go.Scatter(x=results_2.date, y=results_2.arpu, mode="lines+markers", name="arpu")
)
fig.add_trace(
    go.Scatter(
        x=results_2.date,
        y=results_2.arppu,
        mode="lines+markers",
        name="arppu",
    )
)
fig.add_trace(
    go.Scatter(
        x=results_2.date,
        y=results_2.aov,
        mode="lines+markers",
        name="aov",
    )
)
fig.update_layout(
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)
fig.show()

Проаналізуйте побудований графік та спробуйте відповісти на такі питання:

1. Які метрики мають більший розкид значень протягом періоду, що розглядається?
2. Чи можна сказати, що окремі метрики мають аномально високі чи аномально низькі значення окремими днями?
3. Який висновок можна зробити про співвідношення числа користувачів, що платять, і всіх користувачів сервісу в розглянуті дні?

## Задача 3

Доповнимо наш аналіз ще більш цікавими розрахунками — обчислимо ті самі метрики, але для кожного дня враховуватимемо накопичену виручку і всі наявні на даний момент дані про кількість користувачів і замовлень. Таким чином, отримаємо **динамічний ARPU, ARPPU і AOV** і зможемо простежити, як він змінювався протягом часу з урахуванням даних, що надходять нам.

::::: {.callout-note icon=false}
## Завдання
:::: {#exr-sql-metrics-03}
<br>
За таблицями `orders` та `user_actions` для кожного дня розрахуйте такі показники:

1. Накопичений виторг на користувача (**Running ARPU**).
2. Нагромаджений виторг на платить користувача (**Running ARPPU**).
3. Накопичений виторг із замовлення, або середній чек (**Running AOV**).

Колонки з показниками назвіть `running_arpu`, `running_arppu`, `running_aov`. Колонку з датами назвіть `date`.

Під час розрахунку всіх показників округляйте значення **до двох знаків після коми**.

Результат має бути відсортований за зростанням дати.

Поля в результуючій таблиці: `date`, `running_arpu`, `running_arppu`, `running_aov`

**Пояснення:**

При розрахунку числа користувачів та користувачів, що оплатили замовлення на поточну дату, враховуйте відповідних користувачів за всі попередні дні, включаючи поточний.

Користувачами, що оплатили замовлення вважатимемо тих, які на поточний день оформили хоча б одне замовлення, яке надалі не було скасовано.

Вважатимемо, що оплата за замовлення надходить відразу після його оформлення, тобто. Випадки, коли замовлення було оформлено в один день, а оплата отримана наступного, виникнути не можуть.

При розрахунку виручки пам'ятайте, що не всі замовлення були сплачені, деякі були скасовані користувачами.

Не забувайте при діленні заздалегідь приводити значення до потрібного типу даних.

::: {.callout-note collapse="true"}
## Підказка на випадок, якщо зовсім не виходить

Для вирішення цього завдання необхідно доповнити запит з попереднього завдання та для кожного дня додатково розрахувати накопичений виторг, а також накопичену кількість усіх користувачів та оркмо користувачів, які оплатили замовлення. Для розрахунку кількості користувачів з накопиченням буде потрібна інформація про нових користувачів і нових користувачів, які оплатили замовлення у кожен із розглянутих днів.
:::


::::
:::::

In [8]:
#| code-fold: true
#| code-summary: "Рішення"

%%sql
SELECT date, 
        ROUND(SUM(revenue) OVER (ORDER BY date)::decimal / SUM(new_users) OVER (ORDER BY date), 2) AS running_arpu, 
        ROUND(SUM(revenue) OVER (ORDER BY date)::decimal / SUM(new_paying_users) OVER (ORDER BY date), 2) AS running_arppu, 
        ROUND(SUM(revenue) OVER (ORDER BY date)::decimal / SUM(orders) OVER (ORDER BY date), 2) AS running_aov 
    FROM ( 
        SELECT creation_time::date AS date,  
                COUNT(DISTINCT order_id) AS orders,  
                SUM(price) AS revenue 
        FROM ( 
            SELECT order_id, 
                creation_time, 
                UNNEST(product_ids) AS product_id 
            FROM orders 
            WHERE order_id NOT IN (SELECT order_id FROM user_actions WHERE action='cancel_order') 
            ) t1 
        LEFT JOIN products  
        USING(product_id) 
        GROUP BY date 
    ) t2 
    LEFT JOIN ( 
        SELECT time::date AS date, COUNT(DISTINCT user_id) AS users 
        FROM user_actions 
        GROUP BY date 
    ) t3 
    USING (date) 
    LEFT JOIN ( 
        SELECT time::date AS date, COUNT(DISTINCT user_id) AS paying_users 
        FROM user_actions 
        WHERE order_id NOT IN (SELECT order_id FROM user_actions WHERE action='cancel_order') 
        GROUP BY date 
    ) t4 
    USING (date) 
    LEFT JOIN ( 
        SELECT date, COUNT(user_id) AS new_users 
        FROM ( 
            SELECT user_id, MIN(time::date) AS date 
            FROM user_actions 
            GROUP BY user_id 
        ) t5 
        GROUP BY date 
    ) t6 
    USING (date) 
    LEFT JOIN ( 
        SELECT date, COUNT(user_id) AS new_paying_users 
        FROM ( 
            SELECT user_id, MIN(time::date) AS date 
            FROM user_actions 
            WHERE order_id NOT IN (SELECT order_id FROM user_actions WHERE action='cancel_order') 
            GROUP BY user_id 
        ) t7 
        GROUP BY date 
    ) t8 
    USING (date)

,date,running_arpu,running_arppu,running_aov
0,2022-08-24,372.57,393.10,361.77
1,2022-08-25,499.26,517.53,401.66
2,2022-08-26,512.90,530.87,384.10
3,2022-08-27,571.80,590.21,382.99
4,2022-08-28,632.13,649.72,381.08
5,2022-08-29,707.53,726.29,384.24
6,2022-08-30,766.86,786.40,383.14
7,2022-08-31,792.81,813.46,383.49
8,2022-09-01,813.18,832.90,383.11
9,2022-09-02,844.17,863.05,382.77


Збережемо результат запиту у змінну `results_3`:

In [9]:
#| code-fold: true

%%sql
results_3 << SELECT date, 
        ROUND(SUM(revenue) OVER (ORDER BY date)::decimal / SUM(new_users) OVER (ORDER BY date), 2) AS running_arpu, 
        ROUND(SUM(revenue) OVER (ORDER BY date)::decimal / SUM(new_paying_users) OVER (ORDER BY date), 2) AS running_arppu, 
        ROUND(SUM(revenue) OVER (ORDER BY date)::decimal / SUM(orders) OVER (ORDER BY date), 2) AS running_aov 
    FROM ( 
        SELECT creation_time::date AS date,  
                COUNT(DISTINCT order_id) AS orders,  
                SUM(price) AS revenue 
        FROM ( 
            SELECT order_id, 
                creation_time, 
                UNNEST(product_ids) AS product_id 
            FROM orders 
            WHERE order_id NOT IN (SELECT order_id FROM user_actions WHERE action='cancel_order') 
            ) t1 
        LEFT JOIN products  
        USING(product_id) 
        GROUP BY date 
    ) t2 
    LEFT JOIN ( 
        SELECT time::date AS date, COUNT(DISTINCT user_id) AS users 
        FROM user_actions 
        GROUP BY date 
    ) t3 
    USING (date) 
    LEFT JOIN ( 
        SELECT time::date AS date, COUNT(DISTINCT user_id) AS paying_users 
        FROM user_actions 
        WHERE order_id NOT IN (SELECT order_id FROM user_actions WHERE action='cancel_order') 
        GROUP BY date 
    ) t4 
    USING (date) 
    LEFT JOIN ( 
        SELECT date, COUNT(user_id) AS new_users 
        FROM ( 
            SELECT user_id, MIN(time::date) AS date 
            FROM user_actions 
            GROUP BY user_id 
        ) t5 
        GROUP BY date 
    ) t6 
    USING (date) 
    LEFT JOIN ( 
        SELECT date, COUNT(user_id) AS new_paying_users 
        FROM ( 
            SELECT user_id, MIN(time::date) AS date 
            FROM user_actions 
            WHERE order_id NOT IN (SELECT order_id FROM user_actions WHERE action='cancel_order') 
            GROUP BY user_id 
        ) t7 
        GROUP BY date 
    ) t8 
    USING (date)

Візуалізуємо отримані дані:

In [10]:
# | label: fig-sql-analytic-03
# | fig-cap: "Динаміка Running ARPU, Running ARPPU, Running AOV"

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=results_3.date,
        y=results_3.running_arpu,
        mode="lines+markers",
        name="running_arpu",
    )
)
fig.add_trace(
    go.Scatter(
        x=results_3.date,
        y=results_3.running_arppu,
        mode="lines+markers",
        name="running_arppu",
    )
)
fig.add_trace(
    go.Scatter(
        x=results_3.date,
        y=results_3.running_aov,
        mode="lines+markers",
        name="running_aov",
    )
)
fig.update_layout(
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)
fig.show()

Проаналізуйте побудований графік та спробуйте відповісти на такі питання:

1. Яка загалом динаміка у розрахованих метрик? Вони ростуть, падають чи мають приблизно однакове значення у кожен із днів?
2. Чи можна з огляду на динаміку розрахованих метрик припустити, що з часом зростає кількість замовлень на одного користувача?

## Задача 4

Давайте порахуємо ті ж показники, але в іншому розрізі — не просто щодня, а щодня тижня.

Для виділення порядкового номера тижня можна використовувати функцію `DATE_PART` з параметром `'isodow'`.

::::: {.callout-note icon=false}
## Завдання
:::: {#exr-sql-metrics-04}
<br>
Для кожного дня тижня в таблицях `orders` та `user_actions` розрахуйте такі показники:

1. Виручку користувача (ARPU).
2. Виручку на користувача, що платить (ARPPU).
3. Виторг на замовлення (AOV).

При розрахунках враховуйте дані лише за період **з 26 серпня 2022 року до 8 вересня 2022 року включно** — так, щоб до аналізу потрапила однакова кількість усіх днів тижня (рівно по два дні).

У результуючу таблицю включіть як найменування днів тижня (наприклад, Monday), так і порядковий номер дня тижня (від 1 до 7, де 1 Monday, 7 Sunday).

Колонки з показниками назвіть відповідно `arpu`, `arppu`, `aov`. Назвіть колонку з найменуванням дня тижня `weekday`, а колонку з порядковим номером дня тижня `weekday_number`.

Під час розрахунку всіх показників округляйте значення до двох знаків після коми.

Результат має бути відсортований за зростанням порядкового номера дня тижня.

Поля в результуючій таблиці: `weekday`, `weekday_number`, `arpu`, `arppu`, `aov`

**Пояснення:**

У цій задачі порядковий номер дня тижня необхідний для того, щоб дні тижня були розташовані на графіці зліва направо в правильному порядку — не за найменуванням, а за зростанням порядкового номера.

::: {.callout-note collapse="true"}
## Підказка на випадок, якщо зовсім не виходить

Для розв'язання задачі необхідно виконати ті самі операції, що й у попередньому завданні, тільки цього разу для днів тижня. Додатково необхідно правильно задати фільтрацію за датою, щоб у аналіз потрапило рівно два однакові дні тижня.
:::


::::
:::::

In [11]:
#| code-fold: true
#| code-summary: "Рішення"

%%sql
SELECT weekday, t1.weekday_number AS weekday_number, 
        ROUND(revenue::decimal / users, 2) AS arpu, 
        ROUND(revenue::decimal / paying_users, 2) AS arppu, 
        ROUND(revenue::decimal / orders, 2) AS aov 
    FROM ( 
        SELECT DATE_PART('isodow', creation_time) AS weekday, 
                MAX(DATE_PART('isodow', creation_time)) AS weekday_number, 
                COUNT(DISTINCT order_id) AS orders,  
                SUM(price) AS revenue 
        FROM ( 
            SELECT order_id, 
                creation_time, 
                UNNEST(product_ids) AS product_id 
            FROM orders 
            WHERE order_id NOT IN (SELECT order_id FROM user_actions WHERE action='cancel_order') 
                AND creation_time >= '2022-08-26' AND creation_time < '2022-09-09' 
            ) t4 
        LEFT JOIN products  
        USING(product_id) 
        GROUP BY weekday 
    ) t1 
    LEFT JOIN ( 
        SELECT DATE_PART('isodow', time) AS weekday, 
            MAX(DATE_PART('isodow', time)) AS weekday_number, 
            COUNT(DISTINCT user_id) AS users 
        FROM user_actions 
        WHERE time >= '2022-08-26' AND time < '2022-09-09' 
        GROUP BY weekday 
    ) t2 
    USING (weekday) 
    LEFT JOIN ( 
        SELECT DATE_PART('isodow', time) AS weekday,  
            MAX(DATE_PART('isodow', time)) AS weekday_number, 
            COUNT(DISTINCT user_id) AS paying_users 
        FROM user_actions 
        WHERE order_id NOT IN (SELECT order_id FROM user_actions WHERE action='cancel_order') 
            AND time >= '2022-08-26' AND time < '2022-09-09' 
        GROUP BY weekday 
    ) t3 
    USING (weekday) 
    ORDER BY weekday_number

,weekday,weekday_number,arpu,arppu,aov
0,1,1,555.98,575.18,385.87
1,2,2,528.94,548.04,382.63
2,3,3,528.90,548.33,381.16
3,4,4,533.98,551.37,382.62
4,5,5,534.79,553.21,378.70
5,6,6,578.53,595.48,385.74
6,7,7,566.23,583.38,380.48


Збережемо результат запиту у змінну `results_4`:

In [12]:
#| code-fold: true

%%sql
results_4 << SELECT weekday, t1.weekday_number AS weekday_number, 
        ROUND(revenue::decimal / users, 2) AS arpu, 
        ROUND(revenue::decimal / paying_users, 2) AS arppu, 
        ROUND(revenue::decimal / orders, 2) AS aov 
    FROM ( 
        SELECT DATE_PART('isodow', creation_time) AS weekday, 
                MAX(DATE_PART('isodow', creation_time)) AS weekday_number, 
                COUNT(DISTINCT order_id) AS orders,  
                SUM(price) AS revenue 
        FROM ( 
            SELECT order_id, 
                creation_time, 
                UNNEST(product_ids) AS product_id 
            FROM orders 
            WHERE order_id NOT IN (SELECT order_id FROM user_actions WHERE action='cancel_order') 
                AND creation_time >= '2022-08-26' AND creation_time < '2022-09-09' 
            ) t4 
        LEFT JOIN products  
        USING(product_id) 
        GROUP BY weekday 
    ) t1 
    LEFT JOIN ( 
        SELECT DATE_PART('isodow', time) AS weekday, 
            MAX(DATE_PART('isodow', time)) AS weekday_number, 
            COUNT(DISTINCT user_id) AS users 
        FROM user_actions 
        WHERE time >= '2022-08-26' AND time < '2022-09-09' 
        GROUP BY weekday 
    ) t2 
    USING (weekday) 
    LEFT JOIN ( 
        SELECT DATE_PART('isodow', time) AS weekday,  
            MAX(DATE_PART('isodow', time)) AS weekday_number, 
            COUNT(DISTINCT user_id) AS paying_users 
        FROM user_actions 
        WHERE order_id NOT IN (SELECT order_id FROM user_actions WHERE action='cancel_order') 
            AND time >= '2022-08-26' AND time < '2022-09-09' 
        GROUP BY weekday 
    ) t3 
    USING (weekday) 
    ORDER BY weekday_number

Візуалізуємо отримані дані:

In [13]:
# | label: fig-sql-analytic-04
# | fig-cap: "Динаміка ARPU, ARPPU, AOV по дням тижня"

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=[
            "Monday",
            "Tuesday",
            "Wednesday",
            "Thursday",
            "Friday",
            "Saturday",
            "Sunday",
        ],
        y=results_4.arpu,
        mode="lines+markers",
        name="arpu",
    )
)
fig.add_trace(
    go.Scatter(
        x=[
            "Monday",
            "Tuesday",
            "Wednesday",
            "Thursday",
            "Friday",
            "Saturday",
            "Sunday",
        ],
        y=results_4.arppu,
        mode="lines+markers",
        name="arppu",
    )
)
fig.add_trace(
    go.Scatter(
        x=[
            "Monday",
            "Tuesday",
            "Wednesday",
            "Thursday",
            "Friday",
            "Saturday",
            "Sunday",
        ],
        y=results_4.aov,
        mode="lines+markers",
        name="aov",
    )
)
fig.update_layout(
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)
fig.show()

Проаналізуйте побудований графік та спробуйте відповісти на такі питання:

1. У які дні тижня метрики ARPU та ARPPU набували найбільших значень? Як ви вважаєте, чи це узгоджується в цілому зі стандартною поведінкою користувачів сервісу доставки їжі?
2. Як ви вважаєте, чому в ті дні, коли метрики ARPU та ARPPU набували найбільших значень, метрика AOV залишалася приблизно на тому ж рівні? За якого сценарію таке можливе?